In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# A-B test root cause analysis
In this use case we will perform a statistical test, to see if there are significant difference between the periods leading up to incidents (in this case, excessive flaring), and periods sampled from normal operation. If we can identify tags that are behaving differently between both groups, we might identify early indicators notifying us another incident is about to happen, or we could simply use this information to prevent new incidents from occurring.

## Statistical test functions
We will use two functions. The first function tests two groups of intervals (layers) for a single calculation (avg, std, max, ...) on a single tag. The second function iterates over the first for multiple tags and multiple caculations, and ranks the results in a table.

In [2]:
import scipy
import pandas as pd


def kruskal(a, b, tag, operation):
    """Kruskal-Wallis H-test for two sets of intervals for a given tag operation
    
    Tests the null hypothesis that the medians of both populations are equal for the given tag calculation.    
    An example would be to test if average pressure is significantly different between two groups of intervals.
    
    
    Parameters
    ----------
    a : list of Interval
        First population of intervals
    b : list of Interval
        Second population of intervals
    tag : Tag
        Analog or discrete tag on which to perform the calculation to be tested
    operation : str
        Tag operation to be tested
        
    Returns
    -------
    p : float
        p-value for the test
        
    """
    
    # Calculation results
    key = "abtest"
    values = [
        interval[key]
        for interval in tag.calculate(a+b, operation=operation, key=key)
    ]
    
    # Statistical test -> p-value   
    try:
        _ , p = scipy.stats.kruskal(
            [value for value in values[0:len(a)] if not pd.isna(value)], 
            [value for value in values[len(a):] if not pd.isna(value)],
        )
    except:
        p = scipy.nan
    return p


def multitest(a, b, tags, operations):
    """Perform the Kruskal-Wallis H-test for multiple tags and operations
    
    Parameters
    ----------
    a : list of Interval
        First population of intervals
    b : list of Interval
        Second population of intervals
    tags : list of Tag
        Analog or discrete tags on which to perform the calculation to be tested
    operations : list of str
        Tag operation to be tested
        
    Returns
    -------
    df : DataFrame
        p-values per tag and operation, sorted by minimal p-value per tag
        
    """
    
    # Ignore tags with string values
    tags = [tag for tag in tags if tag.isnumeric()]
    
    # Initialize DataFrame
    df = pd.DataFrame(index=[tag.name for tag in tags], columns=[operations])
    
    # Perform tests
    for tag in tags:        
        for operation in operations:
            df.loc[tag.name, operation] = kruskal(a, b, tag, operation)
            
    # Remove NaN rows
    df.dropna(axis=0, how="all", inplace=True)
    
    # Sort results by smallest p-value per tag
    df = df.loc[df.min(axis=1).sort_values().index]
          
    return df

## Authentication

In [3]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"
username = "wdaniels"

client = TrendMinerClient(
    url=url,
    client_id=client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Paris",
)

## Loading the anomalous periods
We can get the anomalous periods by performing a search, getting them from context items, or inserting the dates manually. The most straighforward way in this case, however, is to load them from a view that we have prepared.

In [4]:
# HIDDEN - create view, delete existing views
for view in client.trend.view.by_name("AB RCA"):
    view.delete()

view = client.trend.view(
    name="AB RCA",
    entries=["[CS]TM-PLA-FI020.PV"],
    layers=[
        ("2020-02-03 16:13", "2020-02-03 16:35"),
        ("2019-12-13 12:43", "2019-12-13 13:05"),
        ("2019-10-26 05:04", "2019-10-26 05:26"),
        ("2019-12-29 04:28", "2019-12-29 04:50"),
    ]
)

view.post()

In [5]:
view = client.trend.view.from_name("AB RCA")
anomalies = [layer.interval for layer in view.layers]
anomalies

[<< Interval | 2020-02-03 16:13:00+01:00 | 0:22:00 >>,
 << Interval | 2019-12-13 12:43:00+01:00 | 0:22:00 >>,
 << Interval | 2019-10-26 05:04:00+02:00 | 0:22:00 >>,
 << Interval | 2019-12-29 04:28:00+01:00 | 0:22:00 >>]

We are not particularly interested in the anomalies themselves, however. We know that many things will change once the anomaly (in this case a flow increase) is already happening. It is the period before the anomaly that we want to test to find early indicators. In this case we want to investigate the 15m leading up to the anomalies.

In [6]:
pre_anomalies = [anomaly.before("15m") for anomaly in anomalies]

pre_anomalies

[<< Interval | 2020-02-03 15:58:00+01:00 | 0:15:00 >>,
 << Interval | 2019-12-13 12:28:00+01:00 | 0:15:00 >>,
 << Interval | 2019-10-26 04:49:00+02:00 | 0:15:00 >>,
 << Interval | 2019-12-29 04:13:00+01:00 | 0:15:00 >>]

## Getting normal periods
We sample the normal operating conditions (retrieved by search) for 15m intervals.

In [7]:
import random

flow = client.tag.from_name("[CS]TM-PLA-FI020.PV")

vbs = client.search.value(
    queries=[
        (flow, ">", 20)
    ]
)

normal_operations = vbs.get_results(
    interval=("2019-01-01", "2021-01-01"),
    excluded_intervals=anomalies+pre_anomalies
)

# Set a seed to get the same random results every time
random.seed("abtest")

normal_samples = client.time.interval.sample(
    normal_operations,
    duration="15m",
    n=16,
)

## Statistical test
### Load tags
We need to select the tags on which to run the test. Tags need to be indexed to get results. Here we assume relevant tags are already indexed (otherwise we would batch index them).

In [8]:
# Indexed tags to test
indexes = client.tag.index.by_name("[CS]*", status="OK")
tags = [index.tag for index in indexes]
len(tags)

63

### Run test
We test our tags for significant differences in both average value and standard deviation for the two groups of 15m intevals.

In [9]:
df = multitest(
    pre_anomalies,
    normal_samples,
    tags=tags, 
    operations=["mean", "stdev"],
)

In [10]:
df

mean     stdev
[CS]TM-PLA-XI039                                    0.018163  0.002497
[CS]BA:LEVEL.1_1DAY_AVG_MC                          0.776814  0.037635
[CS]BA:LEVEL.1_1DAY_INTEGRAL                        0.776814  0.037635
[CS]SINUSOID_New                                    0.047221  0.368462
[CS]SINUSOID                                        0.047221  0.368462
[CS]BA:CONC.1                                       0.088973       1.0
[CS]BA:CONC.1+1000/_?*_8($$#                        0.088973       1.0
[CS]BA:CONC.1_custom                                0.088973       1.0
[CS]SINUSOID_10m                                    0.705034  0.118139
[CS]TM-PLA-FI020.PV                                 0.508332  0.185877
[CS]BA:LEVEL.1_1DAY_AVG_test_SINGLE_DATA_POINT_...  0.705034  0.261031
[CS]BA:ACTIVE.1_AVA                                 0.291954  0.291954
[CS]BA:LEVEL.1                                      0.508332  0.395092
[CS]BA:TEMP.1_1DAY_AVG                              0.850107  0.395092
[CS]BA:TEMP.1_1DAY_AVG_test                         0.850107  0.395092
[CS]BA:TEMP.1                                       0.705457  0.449692
[CS]BA:TEMP.1_INTEGRAL_MONTHLY_Vincent+Eduardo+...  0.508332  0.567381
[CS]1003_Pump1_Flow                                 0.636602  0.508332
[CS]SIS2.EX                                         0.921118  0.843006

Only `[CS]TM-PLA-XI039` shows significant differences between both groups, especially in its standard deviation. Since the number of layers is limited, we do a quick visual confirmation in TrendMiner that this tag is indeed significantly different. We do this by generating a url leading us directly to the view.

We could confirm that rapid changes in the state of the valve represented by this tag are indeed the root cause of our anomalies.

In [11]:
view = client.trend.view(
    entries=["[CS]TM-PLA-XI039"], 
    layers=pre_anomalies+normal_samples)

for layer in view.layers[0:len(pre_anomalies)]:
    layer.name = "pre-anomaly"

for layer in view.layers[len(pre_anomalies):]:
    layer.line_style = "dotted"
    layer.name = "normal"
    
view.get_session_url()

'https://cs.trendminer.net/trendhub/#/share/e4a49765d480'